In [1]:
#first it decodes the message and writes it as a text file:
import whisper

model = whisper.load_model("tiny")  # Options: tiny, base, small, medium, large

result = model.transcribe("command_test.mp3")
print(result["text"])

100%|█████████████████████████████████████| 72.1M/72.1M [00:04<00:00, 16.2MiB/s]
C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 test test test I'm gonna be talking like this let's see how it handles when it's super loud and then also when I'm really close and it's super loud or when I'm pretty far away and it's kind of quiet let's see if it can handle a little bit of this a little bit of that some flaps and I think birthday party power off run function meeting at 5 p.m. in new Plaza Square you need a doctor's appointment pretty soon make sure you get that going don't forget to visit the birthday party at 7 p.m.


first lets see if it can do this in real time. 
to do this, since the model is trained on 30 second intervals we will need to break the audio up into 30 second chunks.
then while its transcribing one section, it will be recording the next section, that way its going to be at most 30 seconds behind.
it will save these 30 second chunks to a file, labeled with date and time and then it will search through the file for any new information


In [ ]:
import threading
import os
import numpy as np
import whisper
import sounddevice as sd
from scipy.io.wavfile import write
from pydub import AudioSegment
import time
from datetime import datetime

model = whisper.load_model("medium")
file_lock = threading.Lock()

duration = 30  # seconds
samplerate = 16000
stop_recording = False

recording_buffer = {
    "Thread-1": None,
    "Thread-2": None
}

def record_audio(thread_name):
    print(f"{thread_name}: Recording for {duration}s...")
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    return recording

def save_and_transcribe(thread_name, audio_data):
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    wav_filename = f"{thread_name}_{current_datetime}.wav"
    mp3_filename = f"{thread_name}_{current_datetime}.mp3"

    write(wav_filename, samplerate, audio_data)
    audio = AudioSegment.from_wav(wav_filename)
    audio.export(mp3_filename, format="mp3")
    os.remove(wav_filename)

    print(f"{thread_name}: Transcribing...")
    result = model.transcribe(mp3_filename)
    clean_text = result["text"].replace('\n', ' ').replace('\r', ' ').strip()

    with file_lock:
        with open("transcription.txt", "a", encoding="utf-8") as f:
            f.write(f"[{datetime.now()}][{thread_name}] {clean_text}\n")

    os.remove(mp3_filename)
    print(f"{thread_name}: Transcription complete and file cleaned up.")

def worker(my_name, partner_name):
    global stop_recording
    while not stop_recording:
        # Record audio
        audio_data = record_audio(my_name)
        # Store for the partner to process later (if needed)
        recording_buffer[my_name] = audio_data

        # Allow partner to record while this thread processes previous data
        if recording_buffer[partner_name] is not None:
            save_and_transcribe(partner_name, recording_buffer[partner_name])
            recording_buffer[partner_name] = None
        else:
            # If partner hasn't recorded yet, process own data
            save_and_transcribe(my_name, audio_data)
            recording_buffer[my_name] = None

def start_alternating_threads():
    thread1 = threading.Thread(target=worker, args=("Thread-1", "Thread-2"))
    thread2 = threading.Thread(target=worker, args=("Thread-2", "Thread-1"))

    thread1.start()
    # Stagger the second thread to alternate
    time.sleep(duration)
    thread2.start()

    return thread1, thread2

try:
    thread1, thread2 = start_alternating_threads()
    runtime = 5 * 60  # Run for 5 minutes, adjust as needed
    time.sleep(runtime)
    stop_recording = True
    thread1.join()
    thread2.join()
    print("Recording and transcription stopped.")
except KeyboardInterrupt:
    stop_recording = True
    print("Stopped by user.")


Recorder: Recording audio...
Main: Running...


C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Exception in thread Thread-4 (processing_thread):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.1520.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.1520.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 994, in run
    self._target(*self._args, **s

Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Transcriber: Transcribing audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Queue is full, skipping this recording.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Queue is full, skipping this recording.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Queue is full, skipping this recording.
Recorder: Recording audio...
Main: Interrupted by user.


In [ ]:
#code words and new information
# first it will look throught the transcript for any new information.
# it will do this any time that there is a break in the transcription.
# every 5 minutes it will check through the transcript to see if there is a break with no new text
# if there isnt, then it will look through from the last time it checked, up to the current time.



In [1]:
import threading
import os
import numpy as np
import whisper
import sounddevice as sd
from scipy.io.wavfile import write
from pydub import AudioSegment
import time
from datetime import datetime

model = whisper.load_model("medium")
file_lock = threading.Lock()

duration = 30  # seconds
samplerate = 16000
stop_recording = False

recording_buffer = {
    "Thread-1": None,
    "Thread-2": None
}

def record_audio(thread_name):
    print(f"{thread_name}: Recording for {duration}s...")
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    return recording

def save_and_transcribe(thread_name, audio_data):
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    wav_filename = f"{thread_name}_{current_datetime}.wav"
    mp3_filename = f"{thread_name}_{current_datetime}.mp3"

    write(wav_filename, samplerate, audio_data)
    audio = AudioSegment.from_wav(wav_filename)
    audio.export(mp3_filename, format="mp3")
    os.remove(wav_filename)

    print(f"{thread_name}: Transcribing...")
    result = model.transcribe(mp3_filename)
    clean_text = result["text"].replace('\n', ' ').replace('\r', ' ').strip()

    with file_lock:
        with open("transcription.txt", "a") as f:
            f.write(f"[{datetime.now()}][{thread_name}] {clean_text}\n")

    os.remove(mp3_filename)
    print(f"{thread_name}: Transcription complete and file cleaned up.")

def worker(my_name, partner_name):
    global stop_recording
    while not stop_recording:
        # Record audio
        audio_data = record_audio(my_name)
        # Store for the partner to process later (if needed)
        recording_buffer[my_name] = audio_data

        # Allow partner to record while this thread processes previous data
        if recording_buffer[partner_name] is not None:
            save_and_transcribe(partner_name, recording_buffer[partner_name])
            recording_buffer[partner_name] = None
        else:
            # If partner hasn't recorded yet, process own data
            save_and_transcribe(my_name, audio_data)
            recording_buffer[my_name] = None

def start_alternating_threads():
    thread1 = threading.Thread(target=worker, args=("Thread-1", "Thread-2"))
    thread2 = threading.Thread(target=worker, args=("Thread-2", "Thread-1"))

    thread1.start()
    # Stagger the second thread to alternate
    time.sleep(duration)
    thread2.start()

    return thread1, thread2

try:
    thread1, thread2 = start_alternating_threads()
    runtime = 5 * 60  # Run for 5 minutes, adjust as needed
    time.sleep(runtime)
    stop_recording = True
    thread1.join()
    thread2.join()
    print("Recording and transcription stopped.")
except KeyboardInterrupt:
    stop_recording = True
    print("Stopped by user.")


Thread-1: Recording for 30s...
Thread-2: Recording for 30s...
Thread-1: Transcribing...


C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
T

: 

In [ ]:
import threading
import queue
import sounddevice as sd
import numpy as np
import whisper
from datetime import datetime
import time

# Load Whisper model
model = whisper.load_model("medium")

samplerate = 16000  # 16kHz sample rate
chunk_duration = 10  # seconds per chunk
audio_queue = queue.Queue()
stop_event = threading.Event()
output_file = "transcription-non-thread.txt"

def record_audio():
    """Continuously record audio and push raw audio data with timestamps to the queue."""
    print("Recording started...")
    while not stop_event.is_set():
        recording = sd.rec(int(chunk_duration * samplerate), samplerate=samplerate, channels=1, dtype='float32')
        sd.wait()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        audio_queue.put((recording.copy(), timestamp))
    print("Recording stopped.")

def transcribe_audio():
    """Continuously transcribe audio from the queue and save transcriptions with timestamps."""
    print("Transcription started...")
    while not stop_event.is_set() or not audio_queue.empty():
        try:
            audio_data, timestamp = audio_queue.get(timeout=1)

            # Whisper expects a 1D numpy array
            audio_data = np.squeeze(audio_data)
            result = model.transcribe(audio_data, fp16=False)

            transcription = result['text'].strip()
            if transcription:
                log_entry = f"[{timestamp}] {transcription}\n"
                print(log_entry.strip())
                with open(output_file, 'a') as f:
                    f.write(log_entry)
        except queue.Empty:
            continue
    print("Transcription stopped.")

def main():
    recorder_thread = threading.Thread(target=record_audio)
    transcriber_thread = threading.Thread(target=transcribe_audio)

    recorder_thread.start()
    transcriber_thread.start()

    print("Recording and transcription are running. Press Ctrl+C to stop.")
    try:
        while True:
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("Stopping...")
        stop_event.set()
        recorder_thread.join()
        transcriber_thread.join()
        print("All processes stopped.")

if __name__ == "__main__":
    main()


Recording started...
Transcription started...
Recording and transcription are running. Press Ctrl+C to stop.
[2025-07-19 01:46:34] Okay, so here is a test using transcription that is non-threaded. Let's see how it will work. Let's try it out.
[2025-07-19 01:46:45] for a very long time like that, will it pick it up? Will it crash? What will happen? If I... wait half a second and do that, will it...
[2025-07-19 01:46:55] record what I'm saying or just skip over it, because I think it will skip over it.
[2025-07-19 01:47:06] crazy is it picking it up if it's picking it up that's very impressive i don't know huh if i could
[2025-07-19 01:47:16] 1 2 3 4 5 6 7 8 9 10 11 12 13 14
[2025-07-19 01:47:27] 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 52, 53, 54, 54, 55, 56, 57, 58, 59, 60, 61, 60, 61, 62, 63, 62, 63, 62, 63, 63, 64, 63, 64, 63,
[2025-07-19 01:47:37] 27 28 29 30
[2025-07-1

In [ ]:
#ok so now we need to make it look for key words and information
#first we need to mark the text that has already been read.
#  so i think we add a 1 before the date if its been read
#requirements:
# 1. Read the transcription file line by line.
# 2. get context from across lines to fidn out when there will be meetings.
# 3. figure out what needs to be done. google calender api, alarm, notification, etc.

line = 0

def cointains_keywords(text,keywords):
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return True
    return False

def mark_as_read(line):
    # Mark the line as read by adding a '1' at the start
    return "1" + line[1:]

with open('Transcription.txt', 'r') as file:
    for line_index, line_content in enumerate(file):
        line = line_index
        print(f"Line {line_index}: {line_content.strip()}")

#ok so there are like a million ways to do this. i think i need to ahve a database of 
# keywords and then check if the line contains any of those keywords.
# if it does, i will identify which one (or ones) it contains
# then i will go through one by one and gather the context around it for each keyword. 
# then i will give the context to a gpt model to get the key information out of it
# once i have the key information like the time, date, and what it is for, i will add it 
# to the google calendar, and move onto the next keyword. 
            



Line 0: [2025-07-19 01:55:47] Okay, so I'm gonna make some quick data stuff. I'm gonna say like, oh I'm inviting you to my birthday party
Line 1: [2025-07-19 01:55:58] at 6pm at Newark mall there's gonna be you know people there I'm also thinking about having a meeting at 6pm
Line 2: [2025-07-19 01:56:08] on Thursday the 27th. So make sure you're there for that one. Very important. I also feel like
Line 3: [2025-07-19 01:56:19] should have a meeting to discuss your stocks at 6.15 p.m.
Line 4: [2025-07-19 01:56:29] and also I think we should really have a talk about the event
Line 5: [2025-07-19 01:56:40] to do on the 26th of April 2027 which will be launching our company called Jar...
Line 6: [2025-07-19 01:56:50] com and i think that's good so uh... yeah
Line 7: [2025-07-19 01:57:01] recording
Line 8: [2025-07-19 01:57:11] It is a
